# Initiliation

In [1]:
!pip install transformers datasets --quiet

     |████████████████████████████████| 3.4 MB 4.3 MB/s 
     |████████████████████████████████| 306 kB 65.4 MB/s 
     |████████████████████████████████| 3.3 MB 51.4 MB/s 
     |████████████████████████████████| 895 kB 74.9 MB/s 
     |████████████████████████████████| 596 kB 63.2 MB/s 
     |████████████████████████████████| 61 kB 458 kB/s 
     |████████████████████████████████| 1.1 MB 64.5 MB/s 
     |████████████████████████████████| 132 kB 70.7 MB/s 
     |████████████████████████████████| 243 kB 73.8 MB/s 
     |████████████████████████████████| 271 kB 73.4 MB/s 
     |████████████████████████████████| 160 kB 63.5 MB/s 
     |████████████████████████████████| 192 kB 75.0 MB/s 


In [2]:
from transformers import TrainingArguments
from transformers import Trainer
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, classification_report, confusion_matrix
from datasets import Dataset
from datasets import load_metric

import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import drive

# Data Preparation

In [3]:
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
%cd 'drive/MyDrive/Masterarbeit/Colab Notebooks/OVERVIEW myPers/00_Datasets/Baseline'

/content/drive/.shortcut-targets-by-id/1aHXlqhpj1STohhfU4gn53D4whaLH__Jz/Masterarbeit/Colab Notebooks/OVERVIEW myPers/00_Datasets/Baseline


In [ ]:
dfPJ = pd.read_csv('myPers_NEU_Baseline.csv', sep=",", error_bad_lines=False)
dfPJ

,text,label
0,likes the sound of thunder.,1
1,is so sleepy it's not even funny that's she ca...,1
2,is sore and wants the knot of muscles at the b...,1
3,likes how the day sounds in this new song.,1
4,is home. <3,1
...,...,...
9912,little things give you away.,0
9913,is wishing it was Saturday.,1
9914,is studying hard for the G.R.E.,1
9915,snipers get more head,1


In [ ]:
df_clean= []
for i,row in dfPJ.iterrows():
  df_clean.append({
      'text': str(row['text']),
      'label': int(row['label']),
  })

dfPJ = pd.DataFrame(df_clean)
dfPJ

,text,label
0,likes the sound of thunder.,1
1,is so sleepy it's not even funny that's she ca...,1
2,is sore and wants the knot of muscles at the b...,1
3,likes how the day sounds in this new song.,1
4,is home. <3,1
...,...,...
9912,little things give you away.,0
9913,is wishing it was Saturday.,1
9914,is studying hard for the G.R.E.,1
9915,snipers get more head,1


# Model Training

In [ ]:
modeltype = "princeton-nlp/sup-simcse-bert-base-uncased"

In [ ]:
train, test = train_test_split(dfPJ, test_size=0.2, random_state=0, stratify=dfPJ.label)

train = Dataset.from_pandas(train)
test = Dataset.from_pandas(test)

tokenizer = AutoTokenizer.from_pretrained(modeltype)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_train = train.map(tokenize_function, batched=True)
tokenized_test = test.map(tokenize_function, batched=True)

full_train_dataset = tokenized_train
full_eval_dataset = tokenized_test

model = AutoModelForSequenceClassification.from_pretrained(modeltype, num_labels=2)

training_args = TrainingArguments(
    "SIMCSE_BERT_NEU", 
    evaluation_strategy="epoch",
    save_strategy = 'no',
    save_steps = 100000,
    save_total_limit = 1,
    metric_for_best_model="eval_f1")

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    print(classification_report(labels, preds, labels=[0,1]))
    print(confusion_matrix(labels,preds))
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

Downloading:   0%|          | 0.00/252 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/689 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at princeton-nlp/sup-simcse-bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Hyperparameter Optimization

In [5]:
! pip install optuna --quiet

     |████████████████████████████████| 308 kB 4.3 MB/s 
     |████████████████████████████████| 80 kB 11.1 MB/s 
     |████████████████████████████████| 209 kB 64.3 MB/s 
     |████████████████████████████████| 75 kB 5.2 MB/s 
     |████████████████████████████████| 112 kB 66.5 MB/s 
     |████████████████████████████████| 49 kB 7.3 MB/s 
     |████████████████████████████████| 149 kB 71.7 MB/s 


In [ ]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(modeltype, num_labels=2)

In [ ]:
trainer = Trainer(
      model_init=model_init,
      args=training_args, 
      train_dataset=full_train_dataset, 
      eval_dataset=full_eval_dataset,
      compute_metrics=compute_metrics 
  )

loading configuration file https://huggingface.co/princeton-nlp/sup-simcse-bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/886dba277a27c6ab50ab3d0bfd8839d354cfeed717289623026415c62b687338.1b14bcddba43d86a607eedb4b638b87d30aa00c839358953dbd36f2cd3317c83
Model config BertConfig {
  "_name_or_path": "princeton-nlp/sup-simcse-bert-base-uncased",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.15.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file

In [ ]:
import sklearn.metrics as metrics
import optuna
import sys
import logging

def objective (metrics):
  return metrics['eval_f1']

def hyperparameter_space(trial):

    return {
        "learning_rate": trial.suggest_float("learning_rate", 5e-8, 5e-1, log=True),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [2, 4, 8, 16]),
        "weight_decay": trial.suggest_float("weight_decay", 5e-12, 5e-1, log=True),
        "num_train_epochs": trial.suggest_float("num_train_epochs",1,8,log=True),
        #"adam_epsilon": trial.suggest_float("adam_epsilon", 1e-10, 1e-6, log=True),
        #"seed" : trial.suggest_float("seed",10,60,log=True)
        }

optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = "SIMCSE_BERT_myPers_NEU"  # Unique identifier of the study.
storage_name = "sqlite:///{}.db".format(study_name)

best_run = trainer.hyperparameter_search(hp_space=hyperparameter_space,compute_objective=objective, n_trials=50, direction="maximize",study_name=study_name, storage=storage_name )

study = optuna.create_study()

[I 2021-12-26 07:57:55,081] A new study created in RDB with name: SIMCSE_BERT_myPers_NEU


A new study created in RDB with name: SIMCSE_BERT_myPers_NEU


Trial:
loading configuration file https://huggingface.co/princeton-nlp/sup-simcse-bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/886dba277a27c6ab50ab3d0bfd8839d354cfeed717289623026415c62b687338.1b14bcddba43d86a607eedb4b638b87d30aa00c839358953dbd36f2cd3317c83
Model config BertConfig {
  "_name_or_path": "princeton-nlp/sup-simcse-bert-base-uncased",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.15.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weigh

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.662300,0.670446,0.625000,0.384615,0.312500,0.500000
2,0.666200,0.662101,0.625000,0.384615,0.312500,0.500000
3,0.667900,0.663351,0.625000,0.384615,0.312500,0.500000
4,0.655900,0.661595,0.625000,0.384615,0.312500,0.500000
5,0.660300,0.661855,0.625000,0.384615,0.312500,0.500000
5,0.659100,0.662004,0.625000,0.384615,0.312500,0.500000


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, text.
***** Running Evaluation *****
  Num examples = 1984
  Batch size = 8


...

Trial 42 pruned. 


Trial:
loading configuration file https://huggingface.co/princeton-nlp/sup-simcse-bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/886dba277a27c6ab50ab3d0bfd8839d354cfeed717289623026415c62b687338.1b14bcddba43d86a607eedb4b638b87d30aa00c839358953dbd36f2cd3317c83
Model config BertConfig {
  "_name_or_path": "princeton-nlp/sup-simcse-bert-base-uncased",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.15.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weigh

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.641141,0.632560,0.546814,0.589443,0.558737


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, text.
***** Running Evaluation *****
  Num examples = 1984
  Batch size = 8


              precision    recall  f1-score   support

           0       0.66      0.85      0.74      1240
           1       0.52      0.26      0.35       744

    accuracy                           0.63      1984
   macro avg       0.59      0.56      0.55      1984
weighted avg       0.61      0.63      0.60      1984

[[1059  181]
 [ 548  196]]


In [10]:
import optuna
study_name = "SIMCSE_BERT_myPers_NEU"  # Unique identifier of the study.
storage_name = "sqlite:///{}.db".format(study_name)

In [12]:
storage_name

'sqlite:///SIMCSE_BERT_myPers_NEU.db'

In [13]:
study_name

'SIMCSE_BERT_myPers_NEU'

In [14]:
study = optuna.create_study(study_name=study_name, storage=storage_name, load_if_exists=True, direction="maximize")
df = study.trials_dataframe(attrs=("number", "value", "params", "state"))

[I 2021-12-27 18:09:08,829] Using an existing study with name 'SIMCSE_BERT_myPers_NEU' instead of creating a new one.


In [15]:
df

,number,value,params_learning_rate,params_num_train_epochs,params_per_device_train_batch_size,params_weight_decay,state
0,0,0.384615,1.669565e-04,5.471096,4,1.074384e-06,COMPLETE
1,1,0.384615,4.482892e-04,1.290046,2,8.632204e-11,COMPLETE
2,2,0.384615,8.597617e-04,6.708040,4,2.413819e-09,COMPLETE
3,3,0.384615,6.053976e-07,1.788059,4,5.602703e-06,COMPLETE
4,4,0.577857,4.210435e-05,1.934191,16,4.246199e-02,COMPLETE
5,5,0.593501,1.159442e-05,3.981723,8,1.227235e-03,COMPLETE
6,6,0.384615,1.278789e-04,2.406148,4,3.159995e-05,COMPLETE
7,7,0.384615,2.196175e-03,1.557180,8,1.108643e-11,COMPLETE
8,8,0.384424,1.638421e-06,4.651543,2,1.076484e-10,PRUNED
9,9,0.384615,5.997157e-03,6.378697,8,8.318204e-10,COMPLETE


In [ ]:
fig = optuna.visualization.plot_param_importances(study)
fig.show()

In [ ]:
best_run

In [ ]:
optuna.visualization.plot_intermediate_values(study)

In [ ]:
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_contour(study)

In [ ]:
optuna.visualization.plot_slice(study)

In [ ]:
optuna.visualization.plot_edf(study)